In [92]:
import os
import numpy as np
import itertools

import aocd
from aocd.models import Puzzle
from aocd import submit

import re

In [93]:
current_day = 11
current_year = 2022
puzzle = Puzzle(year=current_year, day=current_day)
puzzle

<Puzzle(2022, 11) at 0x7fbc663a1760 - Monkey in the Middle>

In [94]:
puzzle.input_data.splitlines()

['Monkey 0:',
 '  Starting items: 97, 81, 57, 57, 91, 61',
 '  Operation: new = old * 7',
 '  Test: divisible by 11',
 '    If true: throw to monkey 5',
 '    If false: throw to monkey 6',
 '',
 'Monkey 1:',
 '  Starting items: 88, 62, 68, 90',
 '  Operation: new = old * 17',
 '  Test: divisible by 19',
 '    If true: throw to monkey 4',
 '    If false: throw to monkey 2',
 '',
 'Monkey 2:',
 '  Starting items: 74, 87',
 '  Operation: new = old + 2',
 '  Test: divisible by 5',
 '    If true: throw to monkey 7',
 '    If false: throw to monkey 4',
 '',
 'Monkey 3:',
 '  Starting items: 53, 81, 60, 87, 90, 99, 75',
 '  Operation: new = old + 1',
 '  Test: divisible by 2',
 '    If true: throw to monkey 2',
 '    If false: throw to monkey 1',
 '',
 'Monkey 4:',
 '  Starting items: 57',
 '  Operation: new = old + 6',
 '  Test: divisible by 13',
 '    If true: throw to monkey 7',
 '    If false: throw to monkey 0',
 '',
 'Monkey 5:',
 '  Starting items: 54, 84, 91, 55, 59, 72, 75, 70',
 '  

In [95]:
test_data = '''Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1'''

In [96]:
from collections import deque

class Monkey:
    def __init__(self, monkey_info):
        monkey_info = [line.strip() for line in monkey_info.splitlines()]
        self.monkey_id = int(monkey_info[0].replace('Monkey ','').replace(':',''))
        self.items = deque()
        for item in map(int,monkey_info[1].replace('Starting items: ','').split(', ')):
            self.items.append(item)
        operation = monkey_info[2].split(' = ')[-1]
        self.op_func = lambda old: eval(operation)
        self.op_func.__name__ = operation
        self.test_divisor = int(monkey_info[3].split()[-1])
        self.test_true_monkey_id = int(monkey_info[4].split('monkey ')[-1])
        self.test_false_monkey_id = int(monkey_info[5].split('monkey ')[-1])
        self.num_inspected = 0
        self.monkeys_lcm = 0

    def __repr__(self):
        return f"Monkey {self.monkey_id}: \n" + \
        f"Items = {self.items}\n" + \
        f"Op: {self.op_func}\n" + \
        f"Test: divisible by {self.test_divisor}\n" + \
        f"If true: throw to monkey {self.test_true_monkey_id}\n" + \
        f"If false: throw to monkey {self.test_false_monkey_id}\n" + \
        f"Num inspected = {self.num_inspected}\n"
        
    def turn_part1(self, monkeys):
        while self.items:
            self.num_inspected += 1
            item = self.items.popleft()
            item = self.op_func(item)
            item = item // 3
            if item % self.test_divisor == 0:
                monkeys[self.test_true_monkey_id].items.append(item)
            else:
                monkeys[self.test_false_monkey_id].items.append(item)
        
    def turn_part2(self, monkeys):
        while self.items:
            self.num_inspected += 1
            item = self.items.popleft()
            item = self.op_func(item)
            assert self.monkeys_lcm != 0, "Set monkeys_lcm first!"
            if item > self.monkeys_lcm:
                item = item % self.monkeys_lcm
            if item % self.test_divisor == 0:
                monkeys[self.test_true_monkey_id].items.append(item)
            else:
                monkeys[self.test_false_monkey_id].items.append(item)
        

## Part 1



In [97]:
monkeys = [Monkey(line) for line in test_data.split('\n\n')]
num_monkeys = len(monkeys)

num_iters = 20
for iteration in range(num_iters):
    for i in range(num_monkeys):
        monkeys[i].turn_part1(monkeys)

# print(monkeys)
a, b = sorted([monkey.num_inspected for monkey in monkeys])[-2:]
result = a * b
print(f"{result = }")

result = 10605


In [98]:
monkeys = [Monkey(line) for line in puzzle.input_data.split('\n\n')]
num_monkeys = len(monkeys)

num_iters = 20
for iteration in range(num_iters):
    for i in range(num_monkeys):
        monkeys[i].turn_part1(monkeys)

# print(monkeys)
a, b = sorted([monkey.num_inspected for monkey in monkeys])[-2:]
result = a * b
print(f"{result = }")

result = 56350


In [63]:
puzzle.answer_a = result
puzzle.answer_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


'56350'

## Part 2



In [99]:
from functools import reduce

def lcm(lst):
    return reduce(lambda x, y: x*y, lst, 1)

In [100]:
monkeys = [Monkey(line) for line in test_data.split('\n\n')]
num_monkeys = len(monkeys)
monkeys_lcm = lcm(monkey.test_divisor for monkey in monkeys)
for monkey in monkeys:
    monkey.monkeys_lcm = monkeys_lcm

num_iters = 10000
for iteration in range(num_iters):
    for i in range(num_monkeys):
        monkeys[i].turn_part2(monkeys)

print(monkeys)
a, b = sorted([monkey.num_inspected for monkey in monkeys])[-2:]
result = a * b
print(f"{result = }")

[Monkey 0: 
Items = deque([63602, 56040, 11941, 10573, 61607])
Op: <function Monkey.__init__.<locals>.<lambda> at 0x7fbc660974c0>
Test: divisible by 23
If true: throw to monkey 2
If false: throw to monkey 3
Num inspected = 52166
, Monkey 1: 
Items = deque([90861, 86149, 27648, 21340, 76915])
Op: <function Monkey.__init__.<locals>.<lambda> at 0x7fbc66097790>
Test: divisible by 19
If true: throw to monkey 2
If false: throw to monkey 0
Num inspected = 47830
, Monkey 2: 
Items = deque([])
Op: <function Monkey.__init__.<locals>.<lambda> at 0x7fbc66097820>
Test: divisible by 13
If true: throw to monkey 1
If false: throw to monkey 3
Num inspected = 1938
, Monkey 3: 
Items = deque([])
Op: <function Monkey.__init__.<locals>.<lambda> at 0x7fbc660978b0>
Test: divisible by 17
If true: throw to monkey 0
If false: throw to monkey 1
Num inspected = 52013
]
result = 2713310158


In [101]:
monkeys = [Monkey(line) for line in puzzle.input_data.split('\n\n')]
num_monkeys = len(monkeys)
monkeys_lcm = lcm(monkey.test_divisor for monkey in monkeys)
for monkey in monkeys:
    monkey.monkeys_lcm = monkeys_lcm

num_iters = 10000
for iteration in range(num_iters):
    for i in range(num_monkeys):
        monkeys[i].turn_part2(monkeys)

print(monkeys)
a, b = sorted([monkey.num_inspected for monkey in monkeys])[-2:]
result = a * b
print(f"{result = }")

[Monkey 0: 
Items = deque([1066252, 1066252, 1976518, 6658832, 4712017, 1528869, 2714330])
Op: <function Monkey.__init__.<locals>.<lambda> at 0x7fbc66345dc0>
Test: divisible by 11
If true: throw to monkey 5
If false: throw to monkey 6
Num inspected = 100279
, Monkey 1: 
Items = deque([8986267, 8986267, 8357947, 8986267, 1996207, 628357, 4805257, 4884414, 3397866, 4301076, 4301076])
Op: <function Monkey.__init__.<locals>.<lambda> at 0x7fbc66345d30>
Test: divisible by 19
If true: throw to monkey 4
If false: throw to monkey 2
Num inspected = 66247
, Monkey 2: 
Items = deque([4309322, 6999710, 4135940, 3611178, 2339124, 6658824, 8034800])
Op: <function Monkey.__init__.<locals>.<lambda> at 0x7fbc663451f0>
Test: divisible by 5
If true: throw to monkey 7
If false: throw to monkey 4
Num inspected = 127552
, Monkey 3: 
Items = deque([2860146, 628356, 3880776, 4135939, 6713507, 3611177, 6876712, 5951963])
Op: <function Monkey.__init__.<locals>.<lambda> at 0x7fbc66345820>
Test: divisible by 2
If 

In [91]:
puzzle.answer_b = result
puzzle.answer_b

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 11! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


'13954061248'